## Enhancing Time Series |  Methods & How to 

<b> First let's start by importing the libs we need

In [42]:
from xgboost import XGBRegressor
from prophet import Prophet
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error

<b> For these tests we will be using House Property Sales dataset https://www.kaggle.com/datasets/htagholdings/property-sales?select=raw_sales.csv

In [5]:
df = pd.read_csv('raw_sales.csv')

<b> Let's first check our dataframe and see if we need to make some improvements to the df

In [6]:
df.shape

(29580, 5)

In [8]:
df.dtypes

datesold        object
postcode         int64
price            int64
propertyType    object
bedrooms         int64
dtype: object

<b> Let's correct our time column

In [10]:
df['datesold'] = pd.to_datetime(df['datesold'])

In [11]:
df.sample(2)

,datesold,postcode,price,propertyType,bedrooms
23120,2018-10-31,2906,750000,house,4
20753,2018-01-27,2913,687000,house,4


In [13]:
df['propertyType'].unique()

array(['house', 'unit'], dtype=object)

In [14]:
df['postcode'].nunique()

27

<b> Let's suppose we want a model that can predict price of houses and units.
    <br>     We will start with Prophet

<b> To use prophet correct we need to transform the data column name to ds and the output in y

In [18]:
df.rename(columns = {'datesold' : 'ds', 'price' : 'y'},inplace = True)

In [19]:
df.sample(2)

,ds,postcode,y,propertyType,bedrooms
2268,2010-04-19,2612,1110000,house,4
6792,2013-04-12,2906,391500,house,3


<b> Now, what we also need is to define our train, test and validation.

In [20]:
df['ds'].min()

Timestamp('2007-02-07 00:00:00')

In [21]:
df['ds'].max()

Timestamp('2019-07-27 00:00:00')

<b> Let's use the final 2 months as our validation, and let's do a test with 1 month of data.

In [25]:
train_df = df.loc[df['ds'] < '2019-05-01'].reset_index(drop = True).copy()

In [24]:
test_df = df.loc[df['ds'].between('2019-05-01','2019-05-31')].reset_index(drop = True).copy()

In [23]:
val_df = df.loc[df['ds'] >= '2019-05-31'].reset_index(drop = True).copy()

<b> Now prophet isn't too much of a category friend so if we wanted one model per type of property we would have two models, and if we wanted one model per postal code and type we could have 54 models which is the n of type's * n of postal codes
    <br> let's go simple and go with one model for houses.

In [26]:
ts_model = Prophet()

In [34]:
train_houses = train_df.loc[train_df['propertyType'] == 'house'].reset_index(drop = True).copy()
test_houses = test_df.loc[test_df['propertyType'] == 'house'].reset_index(drop = True).copy()

In [30]:
ts_model.fit(train_houses)

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


<b> From this point on our prophet has been trained, easy! Now we could be enhancing our model by moving the hyperparameters, and comparing against our test! Let's take a glimpse

In [43]:
mean_absolute_error(test_df['y'],ts_model.predict(test_df)['yhat'])

186068.96569862726

In [44]:
ts_model_hyper = Prophet(seasonality_prior_scale=15.0,
                         holidays_prior_scale=20.0,
                         changepoint_prior_scale=0.1)

In [45]:
ts_model_hyper.fit(train_houses)

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [46]:
mean_absolute_error(test_df['y'],ts_model_hyper.predict(test_df)['yhat'])

186189.25730108883

<b> From this point on to enhance with hyperparameters by themselves, the best option would by a Search Grid. So let's validate in our val_df

In [47]:
mean_absolute_error(val_df['y'],ts_model.predict(val_df)['yhat'])

198998.9496292133

In [48]:
mean_absolute_error(val_df['y'],ts_model_hyper.predict(val_df)['yhat'])

199050.64572794279

<b> We get a model, that has an error pretty like our tests, it deprecated just a little bit which is normal but is useful.

In [49]:
############################################

## Let's change to XGBoost

<b> With XGBoost we will have to do a little bit of variable engineering, so let's start by transforming propertyType in bool

In [79]:
df['property_type'] = df['propertyType'].map({'house' : 1, 'unit' : 0})

In [108]:
df_xgb = df.copy()

<b> Now XGBoost won't understand data columns so we have to create the specific columns for each variable

In [109]:
df_xgb['day'] = df_xgb['ds'].dt.day
df_xgb['month'] = df_xgb['ds'].dt.month
df_xgb['year'] = df_xgb['ds'].dt.year

In [110]:
df_xgb.sample(2)

,ds,postcode,y,propertyType,bedrooms,property_type,day,month,year
399,2008-09-30,2614,372000,house,3,1,30,9,2008
3279,2010-12-15,2611,393000,house,3,1,15,12,2010


<b> Now our other biggest step is to create the sliding window, for this test we will be using 28 days as our sliding window

In [111]:
df_xgb.sort_values(['propertyType','postcode','ds'],ignore_index = True,inplace = True)

In [112]:
df_xgb.sample(2)

,ds,postcode,y,propertyType,bedrooms,property_type,day,month,year
19241,2016-10-18,2906,547000,house,4,1,18,10,2016
16762,2013-09-19,2905,451000,house,3,1,19,9,2013


<b> This is tricky, we have to group it by our other "category" variables and then we have to shift without the data so that the data remains on the grouped df

In [118]:
df_xgb['bedrooms_var'] = df_xgb.groupby(['propertyType','postcode'])['bedrooms'].shift(28)

<b> Final steps, let's do the separation by train,test and val

In [119]:
train_df = df_xgb.loc[df_xgb['ds'] < '2019-05-01'].reset_index(drop = True).copy()

In [125]:
train_df = train_df.loc[train_df['bedrooms_var'].notnull()].reset_index(drop = True).copy()

In [126]:
test_df = df_xgb.loc[df_xgb['ds'].between('2019-05-01','2019-05-31')].reset_index(drop = True).copy()

In [127]:
val_df = df_xgb.loc[df_xgb['ds'] >= '2019-05-31'].reset_index(drop = True).copy()

<b> Now as you can see, we don't have to create loads of models, just one XGBoost for all of them!

In [129]:
train_df.sample(2)

,ds,postcode,y,propertyType,bedrooms,property_type,day,month,year,bedrooms_var
14468,2012-05-24,2904,450000,house,4,1,24,5,2012,4.0
25663,2018-09-03,2607,468000,unit,2,0,3,9,2018,2.0


In [140]:
xgb_model = XGBRegressor(n_estimators = 800)

In [141]:
xgb_model.fit(train_df[['property_type','postcode','year','month','day','bedrooms_var']],train_df['y'])

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=800, n_jobs=8,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [142]:
mean_absolute_error(test_df['y'],xgb_model.predict(test_df[['property_type','postcode','year','month','day','bedrooms_var']]))

147371.90397727274

<b> With a little bit more of work we could get out xgb to predict 1 month ahead with 40K less price error than the prophet

<b> From this point on our prophet has been trained, easy! Now we could be enhancing our model by moving the hyperparameters, and comparing against our test! Let's take a glimpse

In [143]:
xgb_model_hyper = XGBRegressor(n_estimators = 1500,learning_rate = 0.003)

In [144]:
xgb_model_hyper.fit(train_df[['property_type','postcode','year','month','day','bedrooms_var']],train_df['y'])

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.003, max_delta_step=0,
             max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=1500, n_jobs=8,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [145]:
mean_absolute_error(test_df['y'],xgb_model_hyper.predict(test_df[['property_type','postcode','year','month','day','bedrooms_var']]))

116829.61022727273

<b> Moving a little bit someparameters we could enhance it even more in test, now let's check in validation

In [146]:
mean_absolute_error(val_df['y'],xgb_model.predict(val_df[['property_type','postcode','year','month','day','bedrooms_var']]))

151434.13351723584

In [147]:
mean_absolute_error(val_df['y'],xgb_model_hyper.predict(val_df[['property_type','postcode','year','month','day','bedrooms_var']]))

125283.40745811856

<b> That's it! I hope you enjoyed the class and the material.
    <br> In case you need any help, i will be happy to help! You can contact me via LinkedIn https://www.linkedin.com/in/leandro-sartini-de-campos-64343915b/